In [1]:
import json
import pandas as pd
import pymorphy2
import numpy as np

In [2]:
train = pd.read_json('train.json')

In [3]:
train['copy_text'] = train.text

In [249]:
train.shape

(1799, 5)

In [441]:
train.extracted_part[0]

{'text': ['Размер обеспечения исполнения контракта 6593.25 Российский рубль'],
 'answer_start': [1279],
 'answer_end': [1343]}

In [440]:
test.head()

,id,text,label,extracted_part
0,762883279,МУНИЦИПАЛЬНЫЙ КОНТРАКТ № ______ на оказание ус...,обеспечение исполнения контракта,{'text': 'Размер обеспечения исполнения контра...
1,311837655,Извещение о проведении электронного аукциона д...,обеспечение исполнения контракта,{'text': 'Размер обеспечения исполнения контра...
2,540954893,Идентификационный код закупки: 222633005300163...,обеспечение исполнения контракта,{'text': 'Размер обеспечение исполнения контра...
3,274660397,Идентификационный код закупки: 222631202689463...,обеспечение исполнения контракта,{'text': 'Размер обеспечение исполнения контра...
4,732742591,Идентификационный код закупки: 222637800031163...,обеспечение исполнения контракта,{'text': 'Размер обеспечение исполнения контра...


In [4]:
import nltk
import re

# My decision

#### В этом задании я использовал суммаризацию текста, т.е. поиск текста по сути.В моей работе я использую экстрактивную суммаризацию  на основе вхождения общих слов, т.к. он заключается в извлечении из исходного текста наиболее «значимых» информационных фраз, которые нам даны. 

In [5]:
# сходство Жакара
def get_jaccard_sim(str1, str2): 
    a = set(str1.split()) 
    b = set(str2.split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [6]:
alphabets= "[А-Я]"
start_index = []
end_index = []
dictonaries = []
digits = "([0-9])"
text = ''
for i in range(train.shape[0]):
    # Я убираю "." между цифрами, чтобы предложения разбивались более точно
    text = re.sub(digits + "[.]" + digits,"\\1 \\2",train['text'][i])
    text = text.split('.')
    # Проанализировав результаты(текста), я заметил, что в нём присутсвуют слова с большой буквы, но эти слова одни и те же.
    #Опустим их для того, чтобы более точно разбивать предложения на фразы
    for example in range(len(text)):
        while re.search(r'\bКонтракта\b', text[example]):
            text[example] = text[example][:text[example].find(re.search(r'\bКонтракта\b', text[example])[0])]+\
            text[example][text[example].find(re.search(r'\bКонтракта\b', text[example])[0])].lower()+\
            text[example][text[example].find(re.search(r'\bКонтракта\b', text[example])[0])+1:]
        while re.search(r'\bРоссийский\b', text[example]):
            text[example] = text[example][:text[example].find(re.search(r'\bРоссийский\b', text[example])[0])]+\
            text[example][text[example].find(re.search(r'\bРоссийский\b', text[example])[0])].lower()+\
            text[example][text[example].find(re.search(r'\bРоссийский\b', text[example])[0])+1:]
        while re.search(r'\bЗаказчику\b', text[example]):
            text[example] = text[example][:text[example].find(re.search(r'\bЗаказчику\b', text[example])[0])]+\
            text[example][text[example].find(re.search(r'\bЗаказчику\b', text[example])[0])].lower()+\
            text[example][text[example].find(re.search(r'\bЗаказчику\b', text[example])[0])+1:]
        while re.search(r'\bДоговора\b', text[example]):
            text[example] = text[example][:text[example].find(re.search(r'\bДоговора\b', text[example])[0])]+\
            text[example][text[example].find(re.search(r'\bДоговора\b', text[example])[0])].lower()+\
            text[example][text[example].find(re.search(r'\bДоговора\b', text[example])[0])+1:]
        while re.search(r'\bЦены\b', text[example]):
            text[example] = text[example][:text[example].find(re.search(r'\bЦены\b', text[example])[0])]+\
            text[example][text[example].find(re.search(r'\bЦены\b', text[example])[0])].lower()+\
            text[example][text[example].find(re.search(r'\bЦены\b', text[example])[0])+1:]
        while re.search(r'\bДесять\b', text[example]):
            text[example] = text[example][:text[example].find(re.search(r'\bДесять\b', text[example])[0])]+\
            text[example][text[example].find(re.search(r'\bДесять\b', text[example])[0])].lower()+\
            text[example][text[example].find(re.search(r'\bДесять\b', text[example])[0])+1:]
        while re.search(r'\bПять\b', text[example]):
            text[example] = text[example][:text[example].find(re.search(r'\bПять\b', text[example])[0])]+\
            text[example][text[example].find(re.search(r'\bПять\b', text[example])[0])].lower()+\
            text[example][text[example].find(re.search(r'\bПять\b', text[example])[0])+1:]
        while re.search(r'\bЗакона\b', text[example]):
            text[example] = text[example][:text[example].find(re.search(r'\bЗакона\b', text[example])[0])]+\
            text[example][text[example].find(re.search(r'\bЗакона\b', text[example])[0])].lower()+\
            text[example][text[example].find(re.search(r'\bЗакона\b', text[example])[0])+1:]
        while re.search(r'\bНМЦК\b', text[example]):
            text[example] = text[example][:text[example].find(re.search(r'\bНМЦК\b', text[example])[0])] + \
            text[example][text[example].find(re.search(r'\bНМЦК\b', text[example])[0]):text[example].find(re.search(r'\bНМЦК\b', text[example])[0])+ len("НМЦК")].lower()+\
            text[example][text[example].find(re.search(r'\bНМЦК\b', text[example])[0])+len("НМЦК"):]
    text = '.'.join(text)
    # Я решил делить фразы не исходя из точек, а исходя из больших букв. Ниже в цикле я расставляю "!" для дальнейшего сплита
    for smbl in range(20, len(text)):
        if text[smbl] in "АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ":
            text = text[:smbl-1] + '!' + text[smbl:]
    # Опускаю слово в нижний регистр для более точной оценки в функции "сходство Жордана"
    while re.search(r'\bОбеспечение\b', text):
        t = text.find(re.search(r'\bОбеспечение\b', text)[0])
        text = text[:t]+text[t].lower()+text[t+1:]
    text = text.split('!')
    koef = {}
    # В этом цикле происходит подсчёт сходства между нашими фразами и исходным лэйблом
    for j in range(len(text)):
        tmp = get_jaccard_sim(text[j],train.label[i])
        if tmp>0:
            koef[text[j]] = tmp
    sorted_values = sorted(koef.values(), reverse=True)
    result = []
    # Анализируя итоговый результат(текст) я заметил, что в любом из них либо имеются символы 'российский рубль',
    # либо '%', либо 'процентов', либо 'процент', поэтому я воспользовался этим, чтобы выбрать наиболее подходящую фразу
    # Также я указал условие, что слов должно быть больше 6 для того, чтобы не выбирать заголовки, а доставать фразы с информацией
    for u in sorted_values:
        for k in koef.keys():
            if koef[k] == u and koef[k] > 0 and len(k.split())>=6 and (k.find('российский рубль') + k.find('%') + k.find('процентов') + k.find('процент'))>0 and "[footnoteRef:" not in k:
                result.append(k)
                break
    # Если фраза, удовлетворяющим условиям не нашлась, то значит её нет в тексте
    if not result:
        decision = ""
        start_index.append(0)
        end_index.append(0)
    # если такая фраза нашлась, то определим её местоположение
    else:
        decision = result[0]
        text = ' '.join(text)
        start_index.append(text.find(result[0]))
        end_index.append((text.find(result[0]) + len(result[0])))
    # создадим словарь с итоговым результатом
    tmp = {}
    tmp['text'] = decision
    tmp['answer_start'] = start_index[i]
    tmp['answer_end'] = end_index[i]
    dictonaries.append(tmp)
    print(f'my answer: {decision}')
    print(f'correct answer: {train.extracted_part[i]["text"][0]}')

my answer: Размер обеспечения исполнения контракта 6593 25 российский рубль
correct answer: Размер обеспечения исполнения контракта 6593.25 Российский рубль
my answer: Поставщик должен предоставить обеспечение исполнения контракта в размере 10 % от цены контракта.
correct answer: Поставщик должен предоставить обеспечение исполнения контракта в размере 10 % от цены Контракта.
my answer: Размер обеспечения исполнения контракта 10 00%
correct answer: Размер обеспечения исполнения контракта 10.00%
my answer: Размер обеспечения исполнения контракта 10 00%
correct answer: Размер обеспечения исполнения контракта 10.00%
my answer: Размер обеспечения исполнения контракта 10 00%
correct answer: Размер обеспечения исполнения контракта 10.00%
my answer: Размер обеспечения исполнения контракта устанавливается в размере 5 (пять) процентов от цены, по которой заключается контракт и составляет _________ (____________) рубля __ копейки.
correct answer: Размер обеспечения исполнения контракта устанавлив

my answer: Поставщик внес обеспечение исполнения контракта в размере 10,00% от цены контракта на сумму _________ рублей. 8 1.1.
correct answer: Принять к сведению, что Поставщик внес обеспечение исполнения контракта в размере 10,00% от цены контракта на сумму _________ рублей.
my answer: Размер обеспечения исполнения контракта 2 00%
correct answer: Размер обеспечения исполнения контракта 2.00%
my answer: Размер обеспечения исполнения контракта 20 00%
correct answer: Размер обеспечения исполнения контракта 20.00%
my answer: обеспечение исполнения контракта установлено в размере 5,00% от цены контракта на сумму _________ рублей.
correct answer: Обеспечение исполнения контракта установлено в размере 5,00% от цены контракта на сумму _________ рублей.
my answer: Размер обеспечения исполнения контракта 1049760 00 российский рубль
correct answer: Размер обеспечения исполнения контракта 1049760.00 Российский рубль
my answer: Поставщик при заключении контракта должен предоставить заказчику обес

my answer: Размер обеспечение исполнения контракта устанавливается от 0,5 % до 30 % начальной (максимальной) цены контракта с учетом особенностей, указанных в части 6 статьи 96 и за исключением случаев, предусмотренных частями 6 1. и 6 2 статьи 96
correct answer: Размер обеспечение исполнения контракта устанавливается от 0,5 % до 30 % начальной (максимальной) цены контракта
my answer: Размер обеспечение исполнения контракта устанавливается от 0,5 % до 30 % начальной (максимальной) цены контракта с учетом особенностей, указанных в части 6 статьи 96 и за исключением случаев, предусмотренных частями 6 1. и 6 2 статьи 96
correct answer: Размер обеспечение исполнения контракта устанавливается от 0,5 % до 30 % начальной (максимальной) цены контракта
my answer: Размер обеспечение исполнения контракта устанавливается от 0,5 % до 30 % начальной (максимальной) цены контракта с учетом особенностей, указанных в части 6 статьи 96 и за исключением случаев, предусмотренных частями 6 1. и 6 2 статьи 9

my answer: Размер обеспечения гарантийных обязательств по контракту предоставляется в размере _______ руб. (1 % от нмцк). 9 2.
correct answer: Размер обеспечения гарантийных обязательств по контракту предоставляется в размере _______ руб. (1 % от НМЦК).
my answer: обеспечение исполнения настоящего контракта установлено в размере 10,00 % от начальной (максимальной) цены контракта. 8 2.
correct answer: Обеспечение исполнения настоящего Контракта установлено в размере 10,00 % от начальной (максимальной) цены Контракта.
my answer: обеспечение исполнения настоящего контракта установлено в размере 10,00 % от начальной (максимальной) цены контракта. 8 2.
correct answer: Обеспечение исполнения настоящего Контракта установлено в размере 10,00 % от начальной (максимальной) цены Контракта.
my answer: Поставщик при заключении контракта должен предоставить заказчику обеспечение исполнения контракта в размере 5 % от цены контракта. 9 2.
correct answer: Поставщик при заключении Контракта должен предо

my answer: Размер обеспечение исполнения контракта устанавливается от 0,5 % до 30 % начальной (максимальной) цены контракта с учетом особенностей, указанных в части 6 статьи 96 и за исключением случаев, предусмотренных частями 6 1. и 6 2 статьи 96
correct answer: Размер обеспечение исполнения контракта устанавливается от 0,5 % до 30 % начальной (максимальной) цены контракта
my answer: Размер обеспечение исполнения контракта устанавливается от 0,5 % до 30 % начальной (максимальной) цены контракта с учетом особенностей, указанных в части 6 статьи 96 и за исключением случаев, предусмотренных частями 6 1. и 6 2 статьи 96
correct answer: Размер обеспечение исполнения контракта устанавливается от 0,5 % до 30 % начальной (максимальной) цены контракта
my answer: Размер обеспечение исполнения контракта устанавливается от 0,5 % до 30 % начальной (максимальной) цены контракта с учетом особенностей, указанных в части 6 статьи 96 и за исключением случаев, предусмотренных частями 6 1. и 6 2 статьи 9

...

correct answer: Размер обеспечения гарантийных обязательств составляет 5550 (Пять тысяч пятьсот пятьдесят) рублей 00 копеек (3 % от начальной (максимальной) цены контракта).
my answer: 
correct answer: 
my answer: 
correct answer: 
my answer: 
correct answer: 
my answer: 
correct answer: 
my answer: обеспечение исполнения договора требуется в размере 10% (десять) процентов от цены, по которой в соответствии с
correct answer: Требования к независимым гарантиям, предоставляемые в качестве обеспечения заявок, гарантийных обязательств и исполнения Договора Обеспечение исполнения Договора требуется в размере 10% (Десять) процентов от цены, по которой в соответствии с Федеральным законом № 44-ФЗ заключается Договор. В случае ес
my answer: 
correct answer: 
my answer: 
correct answer: 
my answer: 
correct answer: 
my answer: 
correct answer: 
my answer: 
correct answer: 
my answer: 
correct answer: 
my answer: 
correct answer: 
my answer: 
correct answer: 
my answer: Размер обеспечения исполн

my answer: 
correct answer: Обеспечение гарантийных обязательств: Не установлено
my answer: 
correct answer: 
my answer: 
correct answer: 
my answer: Сумма обеспечения исполнения гарантийных обязательств предусмотрена в следующем размере: 4 % от цены предложенной победителем закупки,
correct answer: Сумма обеспечения исполнения гарантийных обязательств предусмотрена в следующем размере: 4 % от цены предложенной победителем закупки, НДС не облагается.
my answer: 
correct answer: 
my answer: Размер обеспечения исполнения гарантийных обязательств должен составлять не менее 20 % от начальной (максимальной) цены контракта или максимального значения цены договора (контракта). 10 7.
correct answer: Размер обеспечения исполнения гарантийных обязательств должен составлять не менее 20 % от начальной (максимальной) цены контракта или максимального значения цены договора (контракта).
my answer: Размер обеспечения гарантийных обязательств: 10 % от начальной (максимальной) цены договора, что составл

my answer: 
correct answer: 
my answer: 
correct answer: 
my answer: 
correct answer: 
my answer: обеспечение гарантийных обязательств представляется в размере 10 % от начальной (максимальной) цены договора в сумме__________________ (_____________) рублей. 6 4.
correct answer: Обеспечение гарантийных обязательств представляется в размере 10 % от начальной (максимальной) цены договора в сумме__________________ (_____________) рублей.
my answer: 
correct answer: 
my answer: 
correct answer: 
my answer: 
correct answer: 
my answer: 
correct answer: 
my answer: Сумма обеспечения исполнения гарантийных обязательств предусмотрена в следующем размере: 0,1 % от цены предложенной победителем закупки,
correct answer: Сумма обеспечения исполнения гарантийных обязательств предусмотрена в следующем размере: 0,1 % от цены предложенной победителем закупки, НДС не облагается.
my answer: Подрядчик обязан предоставить заказчику обеспечение гарантийных обязательств на сумму ________ руб. (_______________

my answer: 
correct answer: 
my answer: Размер обеспечения гарантийных обязательств: 1% от начальной (максимальной) цены контракта (договора), что составляет 24 992,34 рублей (двадцать четыре тысячи девятьсот девяносто два рубля 34 копейки).
correct answer: Размер обеспечения гарантийных обязательств: 1% от начальной (максимальной) цены контракта (договора), что составляет 24 992,34 рублей (двадцать четыре тысячи девятьсот девяносто два рубля 34 копейки).
my answer: 
correct answer: 
my answer: 
correct answer: 
my answer: Размер обеспечения гарантийных обязательств установлен в размере 20% от
correct answer: Размер обеспечения гарантийных обязательств установлен в размере 20% от НМЦД: 1 644 839,76 рублей. Гарантийные обязательства обеспечиваются внесением денежных средств участником закупки
my answer: Размер обеспечения гарантийных обязательств установлен в размере 20% от
correct answer: Размер обеспечения гарантийных обязательств установлен в размере 20% от НМЦД: 1 644 839,76 рублей.

my answer: 
correct answer: 
my answer: 
correct answer: 
my answer: 
correct answer: 
my answer: Размер обеспечения исполнения гарантийных обязательств не может превышать 2% начальной (максимальной) цены договора (цены лота). 2 10 4.
correct answer: Размер обеспечения исполнения гарантийных обязательств не может превышать 2% начальной (максимальной) цены договора (цены лота).
my answer: 
correct answer: 
my answer: 
correct answer: 
my answer: Размер обеспечения исполнения гарантийных обязательств не может превышать 2 % начальной (максимальной) цены договора (цены лота) и устанавливается в
correct answer: Размер обеспечения исполнения гарантийных обязательств не может превышать 2 % начальной (максимальной) цены договора (цены лота) и устанавливается в Специальной части.
my answer: Сумма обеспечения гарантийных обязательств предусмотрено в следующем размере: - 2 % от начальной (максимальной) цены договора, что составляет: 213 991,00 руб.,
correct answer: Сумма обеспечения гарантийных о

#### Если вы посмотрите на мой результат и результат train, то вы увидите, что в подавляющем большинстве случаев я правильно определил фразы, но полностью эти фразы могут не совпадать, это связано с тем, что я не достаточно хорошо разделил предложения на фразы. Надеюсь, что на будущей стажировке вы покажете мне, как лучше это сделать, как и многому другому, жду этого с нетерпением.

In [8]:
train.start_index = start_index

C:\Users\Vlaso\AppData\Local\Temp\ipykernel_1012\2938395330.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  train.start_index = start_index


In [9]:
train.result_start_index = train.extracted_part.apply(lambda x: x['answer_start'][0])

C:\Users\Vlaso\AppData\Local\Temp\ipykernel_1012\980958791.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  train.result_start_index = train.extracted_part.apply(lambda x: x['answer_start'][0])


In [10]:
train.dictonaries = dictonaries

C:\Users\Vlaso\AppData\Local\Temp\ipykernel_1012\3010662290.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  train.dictonaries = dictonaries


In [11]:
for i in range(train.shape[0]):
    print(f'{train.start_index[i]} - {train.extracted_part[i]["answer_start"][0]}')

1279 - 1279
1222 - 1222
1297 - 1297
1304 - 1304
1302 - 1302
1256 - 1256
1298 - 1298
1311 - 1311
1302 - 1302
1303 - 1303
1322 - 1322
1305 - 1305
1310 - 1310
1322 - 1322
1310 - 1310
1314 - 1314
1305 - 1305
1280 - 1280
1478 - 1244
1290 - 1290
1279 - 1279
1290 - 1290
1309 - 1309
1279 - 1279
1282 - 1282
1268 - 1268
1300 - 1300
1208 - 1208
1300 - 1300
1220 - 1220
1292 - 1292
1310 - 1310
1318 - 1318
1299 - 1299
1303 - 1303
1292 - 1292
1316 - 1316
1258 - 1258
2143 - 1247
1310 - 1310
1199 - 1199
1320 - 1320
1275 - 1275
1295 - 1295
1323 - 1323
1094 - 1094
1281 - 1281
1303 - 1303
1291 - 1291
1282 - 1282
1303 - 1303
1232 - 1232
1137 - 1137
1279 - 1279
1078 - 1078
1296 - 1272
1330 - 1330
1294 - 1294
1237 - 1237
1254 - 1254
1252 - 1252
1285 - 1285
1297 - 1297
1301 - 1301
1159 - 1159
1300 - 1300
1315 - 1315
1264 - 1264
1140 - 1140
1084 - 1084
1303 - 1303
1308 - 1308
1208 - 1208
1266 - 1266
1010 - 1010
1270 - 1270
1254 - 1254
1297 - 1297
1310 - 1310
1254 - 1254
1249 - 1249
1272 - 1272
1165 - 1165
1299

1260 - 1260
0 - 0
1311 - 1311
0 - 0
0 - 0
1293 - 1293
1269 - 1269
1282 - 1282
1263 - 1263
1014 - 1014
0 - 1259
1084 - 1084
0 - 1264
1265 - 1265
0 - 0
1266 - 1266
0 - 0
1232 - 1232
1014 - 1014
1301 - 1301
0 - 0
1277 - 1277
1317 - 1317
0 - 1271
1312 - 1312
1269 - 1269
1328 - 1328
1268 - 1268
1286 - 1286
1286 - 1286
0 - 1297
1308 - 1308
1231 - 1231
1348 - 1348
1252 - 1252
0 - 1156
1279 - 1279
0 - 0
1275 - 1275
1325 - 1325
0 - 1264
1262 - 1262
0 - 0
1336 - 1336
1076 - 1076
1265 - 1265
1280 - 1280
1308 - 1308
1285 - 1285
1289 - 1289
1284 - 1284
1273 - 1273
1254 - 1254
0 - 1278
1307 - 1307
1279 - 1279
1303 - 1303
1260 - 1260
1298 - 1298
1288 - 1288
1280 - 1280
0 - 1275
1255 - 1255
0 - 1255
1313 - 1313
0 - 0
1261 - 1261
1223 - 1223
1215 - 1215
1273 - 1280
0 - 0
1291 - 1291
1241 - 1241
1281 - 1281
335 - 1243
1228 - 1228
1250 - 1250
1250 - 1250
1305 - 1305
1319 - 1319
0 - 1303
1235 - 1235
1304 - 1304
1276 - 1276
1201 - 1297
1307 - 1307
0 - 0
1286 - 1286
1300 - 1300
791 - 0
1273 - 1273
1291 - 12

In [12]:
from sklearn.metrics import accuracy_score
accuracy_score(train.result_start_index, train.start_index)

0.8371317398554753

# Test dataset

In [ ]:
# Ниже, на тестовой выборке, я использовал тот же код). Я не сделал функцию, т.к. хотел показать вам accuracy_score

In [77]:
test = pd.read_json('test.json')

In [78]:
test.shape

(318, 3)

In [14]:
test.head()

,id,text,label
0,762883279,МУНИЦИПАЛЬНЫЙ КОНТРАКТ № ______ на оказание ус...,обеспечение исполнения контракта
1,311837655,Извещение о проведении электронного аукциона д...,обеспечение исполнения контракта
2,540954893,Идентификационный код закупки: 222633005300163...,обеспечение исполнения контракта
3,274660397,Идентификационный код закупки: 222631202689463...,обеспечение исполнения контракта
4,732742591,Идентификационный код закупки: 222637800031163...,обеспечение исполнения контракта


In [33]:
alphabets= "[А-Я]"
start_index = []
end_index = []
dictonaries = []
digits = "([0-9])"
text = ''
#
for i in range(test.shape[0]):
    text = re.sub(digits + "[.]" + digits,"\\1 \\2",test['text'][i])
    text = text.split('.')
    for example in range(len(text)):
        while re.search(r'\bКонтракта\b', text[example]):
            text[example] = text[example][:text[example].find(re.search(r'\bКонтракта\b', text[example])[0])]+\
            text[example][text[example].find(re.search(r'\bКонтракта\b', text[example])[0])].lower()+\
            text[example][text[example].find(re.search(r'\bКонтракта\b', text[example])[0])+1:]
        while re.search(r'\bРоссийский\b', text[example]):
            text[example] = text[example][:text[example].find(re.search(r'\bРоссийский\b', text[example])[0])]+\
            text[example][text[example].find(re.search(r'\bРоссийский\b', text[example])[0])].lower()+\
            text[example][text[example].find(re.search(r'\bРоссийский\b', text[example])[0])+1:]
        while re.search(r'\bЗаказчику\b', text[example]):
            text[example] = text[example][:text[example].find(re.search(r'\bЗаказчику\b', text[example])[0])]+\
            text[example][text[example].find(re.search(r'\bЗаказчику\b', text[example])[0])].lower()+\
            text[example][text[example].find(re.search(r'\bЗаказчику\b', text[example])[0])+1:]
        while re.search(r'\bДоговора\b', text[example]):
            text[example] = text[example][:text[example].find(re.search(r'\bДоговора\b', text[example])[0])]+\
            text[example][text[example].find(re.search(r'\bДоговора\b', text[example])[0])].lower()+\
            text[example][text[example].find(re.search(r'\bДоговора\b', text[example])[0])+1:]
        while re.search(r'\bЦены\b', text[example]):
            text[example] = text[example][:text[example].find(re.search(r'\bЦены\b', text[example])[0])]+\
            text[example][text[example].find(re.search(r'\bЦены\b', text[example])[0])].lower()+\
            text[example][text[example].find(re.search(r'\bЦены\b', text[example])[0])+1:]
        while re.search(r'\bДесять\b', text[example]):
            text[example] = text[example][:text[example].find(re.search(r'\bДесять\b', text[example])[0])]+\
            text[example][text[example].find(re.search(r'\bДесять\b', text[example])[0])].lower()+\
            text[example][text[example].find(re.search(r'\bДесять\b', text[example])[0])+1:]
        while re.search(r'\bПять\b', text[example]):
            text[example] = text[example][:text[example].find(re.search(r'\bПять\b', text[example])[0])]+\
            text[example][text[example].find(re.search(r'\bПять\b', text[example])[0])].lower()+\
            text[example][text[example].find(re.search(r'\bПять\b', text[example])[0])+1:]
        while re.search(r'\bЗакона\b', text[example]):
            text[example] = text[example][:text[example].find(re.search(r'\bЗакона\b', text[example])[0])]+\
            text[example][text[example].find(re.search(r'\bЗакона\b', text[example])[0])].lower()+\
            text[example][text[example].find(re.search(r'\bЗакона\b', text[example])[0])+1:]
        while re.search(r'\bНМЦК\b', text[example]):
            text[example] = text[example][:text[example].find(re.search(r'\bНМЦК\b', text[example])[0])] + \
            text[example][text[example].find(re.search(r'\bНМЦК\b', text[example])[0]):text[example].find(re.search(r'\bНМЦК\b', text[example])[0])+ len("НМЦК")].lower()+\
            text[example][text[example].find(re.search(r'\bНМЦК\b', text[example])[0])+len("НМЦК"):]
    text = '.'.join(text)
    for smbl in range(20, len(text)):
        if text[smbl] in "АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ":
            text = text[:smbl-1] + '!' + text[smbl:]
    while re.search(r'\bОбеспечение\b', text):
        t = text.find(re.search(r'\bОбеспечение\b', text)[0])
        text = text[:t]+text[t].lower()+text[t+1:]
    text = text.split('!')
    koef = {}
    for j in range(len(text)):
        tmp = get_jaccard_sim(text[j],test.label[i])
        if tmp>0:
            koef[text[j]] = tmp
    sorted_values = sorted(koef.values(), reverse=True)
    result = []
    for u in sorted_values:
        for k in koef.keys():
            if koef[k] == u and koef[k] > 0 and len(k.split())>=6 and (k.find('российский рубль') + k.find('%') + k.find('процентов') + k.find('процент'))>0 and "[footnoteRef:" not in k:
                result.append(k)
                break
    if not result:
        decision = ""
        start_index.append(0)
        end_index.append(0)
    else:
        decision = result[0]
        text = ' '.join(text)
        start_index.append(text.find(result[0]))
        end_index.append((start_index[i] + len(decision)))
    temp = {}
    temp['text'] = [decision]
    temp['answer_start'] = [start_index[i]]
    temp['answer_end'] = [end_index[i]]
    dictonaries.append(temp)
    print(f'my answer: {decision}')
    print(f'text: {test["text"][i]}')

my answer: Размер обеспечения исполнения контракта составляет _______ рублей ____ копеек (5 процентов от цены контракта).
text: МУНИЦИПАЛЬНЫЙ КОНТРАКТ № ______ на оказание услуг по техническому обслуживанию и ремонту принтеров и многофункциональных устройств, заправке и восстановлению картриджей (идентификационный код закупки 223861800296886010100100590019511244) г. Ханты-Мансийск «___» ____________ 2022 г.  (или) возмещения убытков причинённых Исполнителем убытков. 6. Обеспечение исполнения контракта 6.1. Исполнение контракта обеспечиваются предоставлением независимой гарантии, выданной банком и соответствующей требованиям Федерального закона от 05.04.2013 № 44-ФЗ «О контрактной системе в сфере закупок товаров, работ, услуг для обеспечения государственных и муниципальных нужд», или внесением денежных средств на указанный заказчиком счет, на котором в соответствии с законодательством Российской Федерации учитываются операции со средствами, поступающими заказчику. Способ обеспечения исп

my answer: Размер обеспечение исполнения контракта устанавливается от 0,5 % до 30 % начальной (максимальной) цены контракта с учетом особенностей, указанных в части 6 статьи 96 и за исключением случаев, предусмотренных частями 6 1. и 6 2 статьи 96
text: Идентификационный код закупки: 222633008257063300100100240011051244 КОНТРАКТ №____ на поставку продуктов питания город ______________ «___»__________ 20__ год ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ ОБЩЕОБРАЗОВАТЕЛЬНОЕ УЧРЕЖДЕНИЕ САМАРСКОЙ ОБЛАСТИ СРЕДНЯЯ ОБЩЕОБРАЗОВАТЕЛЬНАЯ ШКОЛА № 1 ''ОБРАЗОВАТЕЛЬНЫЙ ЦЕНТР'' П.Г.Т. СМЫШЛЯЕВКА  который является казенным учреждением; 2) осуществления закупки услуги по предоставлению кредита; 3) заключения бюджетным учреждением, государственным, муниципальным унитарными предприятиями контракта, предметом которого является выдача независимой гарантии. Заказчик вправе установить, что настоящий раздел не применяется в случае заключения Контракта по результатам проведения электронного запроса котировок (за исключением слу

my answer: Размер обеспечения исполнения контракта устанавливается в размере 30 % от начальной (максимальной) цены контракта и составляет 15 399 рублей 00 копеек.
text: Приложение 1 к приказу Администрации Губернатора и Правительства Орловской области от ________________ 2022 г. № ______ ИЗВЕЩЕНИЕ О ПРОВЕДЕНИИ ЭЛЕКТРОННОГО АУКЦИОНА на определение исполнителя для осуществления закупки услуг для обеспечения занием предмета запроса, но без указания УЗ, от которого поступил указанный запрос, при условии, что указанный запрос поступил заказчику не позднее чем за три дня до окончания срока подачи заявок на участие в аукционе. Дата начала предоставления УЗ разъяснений положений извещения: 08 сентября 2022 г. Дата окончания срока предоставления УЗ разъяснений положений извещения: 14 сентября 2022 г. 1.16 Дата и время окончания срока подачи заявок на участие в аукционе 16 сентября 2022 г. 09 часов 00 минут по московскому времени. 1.17 Дата проведения процедуры подачи предложений о цене контракт

my answer: В случае, если предложенные в заявке участника закупки цена, сумма цен единиц товара, работы, услуги снижены на 25 и более процентов по отношению к начальной (максимальной) цене контракта, начальной сумме цен единиц товара, работы, услуги, участник закупки, с которым заключается контракт, предоставляет обеспечение исполнения контракта с учетом положений статьи 37 закона №44
text: Версия с 01.07.2022 года Государственное казенное учреждение Республики Саха (Якутия) ''Республиканский центр пищевых технологий Республики Саха (Якутия)'' Подписано усиленной электронной подписью ИНФОРМАЦИОННАЯ КАРТА К ИЗВЕЩЕНИЮ ОБ ОСУЩЕСТВЛЕНИИ ЗАКУПКИ (в электронной сумма цен таких контрактов должна составлять не менее начальной (максимальной) цены контракта, указанной в извещении. В случае, если предложенные в заявке участника закупки цена, сумма цен единиц товара, работы, услуги снижены на 25 и более процентов по отношению к начальной (максимальной) цене контракта, начальной сумме цен единиц то

my answer: Сумма обеспечения исполнения договора предусмотрена в следующем размере: 5 % от
text: PAGE УТВЕРЖДАЮ ГОСУДАРСТВЕННОЕ АВТОНОМНОЕ УЧРЕЖДЕНИЕ ГОРОДА МОСКВЫ «ГОРМЕДТЕХНИКА ДЕПАРТАМЕНТА ЗДРАВООХРАНЕНИЯ ГОРОДА МОСКВЫ» Т.В. Чечкова « » 2022 г. М. П. АУКЦИОННАЯ ДОКУМЕНТАЦИЯ для субъектов малого и среднего предпринимательства Аукцион в  принять решение об отказе от проведения аукциона до наступления даты и времени окончания срока подачи заявок на участие в аукционе. 6 Размер и порядок внесения денежных средств, в качестве обеспечения заявок на участие в закупке и обеспечения договора 6.1 Размер обеспечения заявок на участие в аукционе (лоте) 0% от начальной (максимальной) цены договора 6.2 Способы внесения обеспечения заявки. Срок и порядок внесения обеспечения заявки Обеспечение заявки может быть оформлено в виде безотзывной банковской гарантии или внесения денежных средств на специальный счет, открытый им в банке, включенном в перечень, определенный Правительством Российской Федера

...

my answer: Размер обеспечения гарантийных обязательств 630263 76 российский рубль
text: Извещение о проведении электронного аукциона для закупки №0832200006622001027 Общая информация Номер извещения 0832200006622001027 Наименование объекта закупки Устройство линий освещения на автомобильных дорогах общего пользования регионального и межмуниципального значения в Шарангском я исполнения контракта, требования к обеспечению В соответствии с требованиями части 14.1 раздела II Извещения об осуществлении закупки Платежные реквизиты "Номер расчётного счёта" 03222643220000003200 "Номер лицевого счёта" 403066020030 "БИК" 012202102 "Наименование кредитной организации" Волго-Вятское ГУ Банка России г.Нижний Новгород "Номер корреспондентского счета" Требования к гарантии качества товара, работы, услуги Требуется гарантия качества товара, работы, услуги Да Информация о требованиях к гарантийному обслуживанию товара В соответствии с требованиями пунктом 2) части 4.1 раздела II Извещения об осуществле

my answer: Размер обеспечения гарантийных обязательств 44661 20 российский рубль
text: Извещение о проведении электронного аукциона для закупки №0816500000622014187 Общая информация Номер извещения 0816500000622014187 Наименование объекта закупки Выполнение работ по ремонту пола центрального входа поликлиники Способ определения поставщика (подрядчика, исполнителя) Электронный ие заявок не требуется Обеспечение исполнения контракта Требуется обеспечение исполнения контракта Размер обеспечения исполнения контракта 5.00% Порядок обеспечения исполнения контракта, требования к обеспечению В соответствии с пунктом 7 части 4 информационной карты к извещению Платежные реквизиты "Номер расчётного счёта" 03224643980000001601 "Номер лицевого счёта" 20054035039 "БИК" 019805001 "Наименование кредитной организации" ОТДЕЛЕНИЕ-НБ РЕСПУБЛИКА САХА (ЯКУТИЯ) БАНКА РОСИИ/УФК по Республике Саха (Якутия) г.Якутск "Номер корреспондентского счета" 40102810345370000085 Требования к гарантии качества товара, раб

my answer: Размер обеспечения гарантийных обязательств 16779 51 российский рубль
text: Извещение о проведении электронного аукциона для закупки №0158300002222000076 Общая информация Номер извещения 0158300002222000076 Наименование объекта закупки «Приобретение жилья для обеспечения жилым помещением детей-сирот и детей, оставшихся без попечения родителей, лиц 016015102 "Наименование кредитной организации" Отделение Ростов-на-Дону//УФК по Ростовской области "Номер корреспондентского счета" 40102810845370000050 Требования к гарантии качества товара, работы, услуги Требуется гарантия качества товара, работы, услуги Да Информация о требованиях к гарантийному обслуживанию товара Поставщик несет ответственность за недостатки (дефекты), обнаруженные в период гарантийного срока, если не докажет, что они произошли вследствие нормального износа жилого помещения, неправильной эксплуатации, ненадлежащего ремонта, произведенного Заказчиком или привлеченными Заказчиком третьими лицами. Требования к г

my answer: обеспечение исполнения гарантийных обязательств предусмотренных договором составляет: 5 % от цены договора.
text: УТВЕРЖДАЮ ГОСУДАРСТВЕННОЕ АВТОНОМНОЕ УЧРЕЖДЕНИЕ ЗДРАВООХРАНЕНИЯ ГОРОДА МОСКВЫ «СТОМАТОЛОГИЧЕСКАЯ ПОЛИКЛИНИКА № 5» ДЕПАРТАМЕНТА ЗДРАВООХРАНЕНИЯ ГОРОДА МОСКВЫ ________________/ А.Н.Валиева М.П. ДОКУМЕНТАЦИЯ КОНКУРСА В ЭЛЕКТРОННОЙ ФОРМЕ, УЧАСТНИКАМИ КОТОРОГО МОГУТ БЫТЬ ТОЛЬКО СУБЪЕКТЫ МАЛОГО 004525988 Особенности внесения: договор заключается после предоставления заказчику обеспечения исполнения договора. Обеспечение исполнения договора может быть представлено в виде безотзывной независимой гарантии или внесением денежных средств на указанный заказчиком счет, на котором в соответствии с законодательством Российской Федерации учитываются операции со средствами, поступающими заказчику, в размере обеспечения исполнения договора, указанном в документации о закупках. Безотзывная независимая гарантия должна соответствовать требованиям, установленным Гражданским кодексом Р

my answer: 
text: Версия с 04.07.2022 года У Т В Е Р Ж Д А Ю «Государственное автономное учреждение Республики Саха (Якутия) ''Республиканская больница №1 - Национальный центр медицины''» Подписано усиленной квалифицированной электронной ентр медицины'' Российская Федерация, 677000, Саха /Якутия/ Респ, Якутск г, СЕРГЕЛЯХСКОЕ, 4, 677010, строение 4, zakazncm@mail.ru, 7-4112-395001 21. Контактное лицо заказчика. ФИО, телефон, электронная почта Рязанцева Марина Алексеевна, 8-4112-395860, zakazncm@mail.ru 22. Банковские реквизиты счета заказчика, на котором в соответствии с законодательством РФ учитываются операции со средствами, поступающими заказчику. Государственное автономное учреждение Республики Саха (Якутия) ''Республиканская больница №1 - Национальный центр медицины'' ИНН: 1435099188 БИК: 019805001 Расчетный счет: 03224643980000001601 Лицевой счет: 30054035069 23. Источник финансирования. Средства, полученные на оказание и оплату медицинской помощи по обязательному медицинскому стр

In [34]:
test['extracted_part'] = dictonaries

In [35]:
test.head()

,id,text,label,extracted_part
0,762883279,МУНИЦИПАЛЬНЫЙ КОНТРАКТ № ______ на оказание ус...,обеспечение исполнения контракта,{'text': ['Размер обеспечения исполнения контр...
1,311837655,Извещение о проведении электронного аукциона д...,обеспечение исполнения контракта,{'text': ['Размер обеспечения исполнения контр...
2,540954893,Идентификационный код закупки: 222633005300163...,обеспечение исполнения контракта,{'text': ['Размер обеспечение исполнения контр...
3,274660397,Идентификационный код закупки: 222631202689463...,обеспечение исполнения контракта,{'text': ['Размер обеспечение исполнения контр...
4,732742591,Идентификационный код закупки: 222637800031163...,обеспечение исполнения контракта,{'text': ['Размер обеспечение исполнения контр...


In [ ]:
### Вывод датасета

In [36]:
dataset = test

In [70]:
result = dataset.to_json(force_ascii=False) 

In [71]:
with open("predictions.json", 'w',encoding="utf-8") as f:
    f.write(result)

In [72]:
example = pd.read_json('predictions.json')

In [76]:
test.shape, example.shape

((318, 4), (318, 4))